In [ ]:
###########################neuron + soma-axis block-v1#################################################

In [ ]:
# %%
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, meijering, threshold_local
import napari
import pandas as pd
import imageio
import cv2

from skimage.exposure import rescale_intensity
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, skeletonize_3d
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops

# --------------------------
# Helpers
# --------------------------
def safe_minmax_norm(volume):
    vol = volume.astype(np.float32)
    vmin, vmax = float(vol.min()), float(vol.max())
    if vmax > vmin:
        vol = (vol - vmin) / (vmax - vmin)
    else:
        vol[:] = 0.0
    return vol

def to_uint8_stack(volume):
    vol = safe_minmax_norm(volume)
    return (vol * 255).astype(np.uint8)

def in_bounds(z, y, x, shape):
    return (0 <= z < shape[0]) and (0 <= y < shape[1]) and (0 <= x < shape[2])

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image  = img_ch1       # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (soma + neurites)

# --------------------------
# 2) SMOOTHING (CH1)
# --------------------------
image_smooth = gaussian(image, sigma=1)

# --------------------------
# 3) BLOB DETECTION (CH1)
# --------------------------
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)
if len(blobs) > 0:
    # convert sigma->radius for LoG blobs (3D)
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)
print(f"Detected {len(blobs)} lysosomes.")

# --------------------------
# 4) REGIONAL QUANTIFICATION (CH1) on a 3D GRID (kept; optional)
# --------------------------
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.digitize(blobs[:, 0], z_bins) - 1
    y_idx = np.digitize(blobs[:, 1], y_bins) - 1
    x_idx = np.digitize(blobs[:, 2], x_bins) - 1
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})

if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("radius_voxels", lambda r: np.sum(4/3 * np.pi * r**3))
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 5) NAPARI VIEWER (we’ll overlay Ch1+Ch2 later)
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# --------------------------
# 6) NEURON SEGMENTATION (neuron = soma + axis)
# --------------------------
# Normalize and denoise Ch2
ch2 = rescale_intensity(image_2.astype(np.float32), in_range='image', out_range=(0,1))
ch2 = gaussian(ch2, sigma=0.6, preserve_range=True)

# (i) Ridge response for thin processes (neurites/axis)
ridge = meijering(ch2, sigmas=np.linspace(0.6, 2.0, 8), black_ridges=False)

# (ii) Base NEURON mask (soma + processes) via local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    # Tune block_size (odd: 51–81) and offset (-0.4..-0.8)*std for your data
    t = threshold_local(R, block_size=61, offset=-0.5*np.std(R))
    neuron_mask[z] = R > t

# cleanup neuron mask
neuron_mask = remove_small_objects(neuron_mask, min_size=8000, connectivity=5)
neuron_mask = binary_closing(neuron_mask, ball(2))

# (iii) AXIS mask proposal from ridge + local threshold
axis_mask_ridge = np.zeros_like(ridge, dtype=bool)
for z in range(ridge.shape[0]):
    Rt = threshold_local(ridge[z], block_size=31, offset=-0.5*np.std(ridge[z]))
    axis_mask_ridge[z] = ridge[z] > Rt

axis_mask_ridge = remove_small_objects(axis_mask_ridge, min_size=200, connectivity=3)
axis_mask_ridge = binary_opening(axis_mask_ridge, ball(1))
axis_mask_ridge = binary_closing(axis_mask_ridge, ball(1))

# Restrict axis to inside neuron
axis_mask = axis_mask_ridge & neuron_mask

# (iv) SOMA via thickness (distance transform): voxels with radius >= threshold
dist = edt(neuron_mask)
soma_min_radius_vox = 3  # <-- key knob; increase (e.g., 4–6) if soma is thicker
soma_mask = (dist >= soma_min_radius_vox)
soma_mask &= neuron_mask
soma_mask = binary_opening(soma_mask, ball(1))
soma_mask = binary_closing(soma_mask, ball(2))

# (v) Make axis exclusive of soma (final split)
axis_mask = neuron_mask & ~soma_mask

# (vi) Optional: skeleton of neurites
axis_skel = skeletonize_3d(axis_mask)

# (vii) SOMA SHAPE CLASSIFICATION (2D eccentricity on the slice with max soma area)
soma_shape = "none"
soma_cc_lab = label(soma_mask)
if soma_cc_lab.max() > 0:
    props3d = regionprops(soma_cc_lab)
    props3d.sort(key=lambda p: p.area, reverse=True)
    soma_labval = props3d[0].label

    # pick z slice with maximum soma area
    z_areas = [np.sum(soma_cc_lab[z] == soma_labval) for z in range(soma_cc_lab.shape[0])]
    z_best = int(np.argmax(z_areas))

    soma_2d = (soma_cc_lab[z_best] == soma_labval).astype(np.uint8)
    rp2 = regionprops(soma_2d)
    if rp2:
        ecc = rp2[0].eccentricity  # 0≈circle, →1 elongated
        soma_shape = "circular" if ecc < 0.6 else "elliptical"
    else:
        soma_shape = "unknown"

print(f"Soma shape classified as: {soma_shape}")

# --------------------------
# 7) Map lysosomes to neuron/soma/axis and save summaries
# --------------------------
in_neuron = []
in_soma = []
in_axis = []
loc_label = []

if len(blobs) > 0:
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not in_bounds(zz, yy, xx, neuron_mask.shape):
            in_neuron.append(False); in_soma.append(False); in_axis.append(False)
            loc_label.append("outside")
            continue
        n = bool(neuron_mask[zz, yy, xx])
        s = bool(soma_mask[zz, yy, xx])
        a = bool(axis_mask[zz, yy, xx])
        in_neuron.append(n)
        in_soma.append(s)
        in_axis.append(a)
        loc_label.append("soma" if s else ("axis" if a else ("neuron_other" if n else "outside")))

if len(df) > 0:
    df["in_neuron_ch2"] = in_neuron
    df["in_soma_ch2"]   = in_soma
    df["in_axis_ch2"]   = in_axis
    df["location_ch2"]  = loc_label
    df["soma_shape"]    = soma_shape

    # overall counts
    counts = pd.Series({
        "total_lysosomes": len(df),
        "in_neuron": int(np.sum(df["in_neuron_ch2"])),
        "in_soma":   int(np.sum(df["in_soma_ch2"])),
        "in_axis":   int(np.sum(df["in_axis_ch2"])),
    })
    counts_df = counts.to_frame(name="value")
    counts_df.to_csv("lysosome_location_summary.csv")

    # per-class counts
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_by_location.csv", index=False)

    # detailed table
    df.to_csv("lysosomes_with_neuron_location.csv", index=False)

print("Saved: lysosome_location_summary.csv, lysosome_counts_by_location.csv, lysosomes_with_neuron_location.csv")

# --------------------------
# 8) Visualization: soma & axis overlays + colored lysosomes
# --------------------------
# Soma (red)
soma_layer = viewer.add_labels(soma_mask.astype(np.uint8), name='Soma (Ch2)', opacity=0.35)
try:
    soma_layer.color = {1: (1.0, 0.0, 0.0, 1.0)}  # red
except Exception:
    pass
soma_layer.blending = 'translucent_no_depth'

# Axis/neurites (green)
axis_layer = viewer.add_labels(axis_mask.astype(np.uint8), name='Axis (Ch2)', opacity=0.35)
try:
    axis_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
axis_layer.blending = 'translucent_no_depth'

# Optional: skeleton overlay (thin gray/white)
viewer.add_image(
    axis_skel.astype(np.uint8),
    name='Axis skeleton',
    blending='additive',
    contrast_limits=(0, 1),
    colormap='gray',
    opacity=0.8
)

# Lysosomes colored by location (orange=soma, cyan=axis, white=outside, gray=neuron_other)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist())
    colors = np.zeros((len(loc), 4), dtype=float)
    colors[loc == "soma"]         = [1.0, 0.5, 0.0, 1.0]  # orange
    colors[loc == "axis"]         = [0.0, 1.0, 1.0, 1.0]  # cyan
    colors[loc == "outside"]      = [1.0, 1.0, 1.0, 1.0]  # white
    colors[loc == "neuron_other"] = [0.6, 0.6, 0.6, 1.0]  # gray

    pts = viewer.add_points(
        blobs[:, :3],
        size=np.clip(blobs[:, 3] * 2, 2, None),
        face_color=colors,
        name='Lysosomes (orange=soma, cyan=axis)'
    )
    # Add text labels robustly (older/newer napari)
    try:
        pts.properties = {
            'lys_id': df['id'].to_numpy(),
            'where':  df['location_ch2'].to_numpy()
        }
        pts.text = {'text': '{where}', 'size': 8, 'color': 'white', 'anchor': 'upper left'}
        # Optional edges (set after creation for older napari)
        pts.edge_color = 'black'
        pts.edge_width = 0.3
    except Exception:
        pass

# Optional: 3D view
# viewer.dims.ndisplay = 3

# --------------------------
# 9) (Optional) Quick videos
# --------------------------
# Fused overlay (Ch2 base + soma/axis + lysosome circles)
img_norm_2 = to_uint8_stack(image_2)
frames_fused = []
for z in range(img_norm_2.shape[0]):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)

    # Soma in red, Axis in green (simple color overlays)
    soma = (soma_mask[z].astype(np.uint8) * 255)
    axis = (axis_mask[z].astype(np.uint8) * 255)
    overlay = base.copy()
    # add red (R channel) for soma
    overlay[..., 2] = np.maximum(overlay[..., 2], soma)
    # add green (G channel) for axis
    overlay[..., 1] = np.maximum(overlay[..., 1], axis)
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    # Draw lysosomes by location
    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            # Determine location at this exact voxel
            if in_bounds(z, y, x, soma_mask.shape) and soma_mask[z, y, x]:
                color = (0, 128, 255)  # BGR orange-ish
            elif in_bounds(z, y, x, axis_mask.shape) and axis_mask[z, y, x]:
                color = (255, 255, 0)  # BGR cyan-ish
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_neuron_soma_axis.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_neuron_soma_axis.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_neuron_soma_axis.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_neuron_soma_axis.gif")

# --------------------------
# 10) RUN VIEWER
# --------------------------
# Put soma shape in the window title (nice reminder)
try:
    viewer.title = f"Neuron segmentation — soma shape: {soma_shape}"
except Exception:
    pass

napari.run()

In [ ]:
###########################neuron + soma-axis block-version 2#################################################

In [ ]:
# %%
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, meijering, threshold_local
import napari
import pandas as pd
import imageio
import cv2

from skimage.exposure import rescale_intensity
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, skeletonize_3d
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops

# --------------------------
# Helpers
# --------------------------
def safe_minmax_norm(volume):
    vol = volume.astype(np.float32)
    vmin, vmax = float(vol.min()), float(vol.max())
    if vmax > vmin:
        vol = (vol - vmin) / (vmax - vmin)
    else:
        vol[:] = 0.0
    return vol

def to_uint8_stack(volume):
    vol = safe_minmax_norm(volume)
    return (vol * 255).astype(np.uint8)

def in_bounds(z, y, x, shape):
    return (0 <= z < shape[0]) and (0 <= y < shape[1]) and (0 <= x < shape[2])

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image  = img_ch1       # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (soma + neurites)

# --------------------------
# 2) SMOOTHING (CH1)
# --------------------------
image_smooth = gaussian(image, sigma=1)

# --------------------------
# 3) BLOB DETECTION (CH1)
# --------------------------
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)
if len(blobs) > 0:
    # convert sigma->radius for LoG blobs (3D)
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)
print(f"Detected {len(blobs)} lysosomes.")

# --------------------------
# 4) REGIONAL QUANTIFICATION (CH1) on a 3D GRID (kept; optional)
# --------------------------
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.digitize(blobs[:, 0], z_bins) - 1
    y_idx = np.digitize(blobs[:, 1], y_bins) - 1
    x_idx = np.digitize(blobs[:, 2], x_bins) - 1
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})

if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("radius_voxels", lambda r: np.sum(4/3 * np.pi * r**3))
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 5) NAPARI VIEWER (we’ll overlay Ch1+Ch2 later)
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# --------------------------
# 6) NEURON SEGMENTATION (neuron = soma + axis)
# --------------------------
# Normalize and denoise Ch2
ch2 = rescale_intensity(image_2.astype(np.float32), in_range='image', out_range=(0,1))
ch2 = gaussian(ch2, sigma=0.6, preserve_range=True)

# (i) Ridge response for thin processes (neurites/axis)
ridge = meijering(ch2, sigmas=np.linspace(0.6, 2.0, 8), black_ridges=False)

# (ii) Base NEURON mask (soma + processes) via local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    # Tune block_size (odd: 51–81) and offset (-0.4..-0.8)*std for your data
    t = threshold_local(R, block_size=61, offset=-0.5*np.std(R))
    neuron_mask[z] = R > t

# cleanup neuron mask
neuron_mask = remove_small_objects(neuron_mask, min_size=8000, connectivity=5)
neuron_mask = binary_closing(neuron_mask, ball(2))

# (iii) AXIS mask proposal from ridge + local threshold
axis_mask_ridge = np.zeros_like(ridge, dtype=bool)
for z in range(ridge.shape[0]):
    Rt = threshold_local(ridge[z], block_size=31, offset=-0.5*np.std(ridge[z]))
    axis_mask_ridge[z] = ridge[z] > Rt

axis_mask_ridge = remove_small_objects(axis_mask_ridge, min_size=200, connectivity=3)
axis_mask_ridge = binary_opening(axis_mask_ridge, ball(1))
axis_mask_ridge = binary_closing(axis_mask_ridge, ball(1))

# Restrict axis to inside neuron
axis_mask = axis_mask_ridge & neuron_mask

# (iv) SOMA via thickness (distance transform): voxels with radius >= threshold
dist = edt(neuron_mask)
soma_min_radius_vox = 3  # <-- key knob; increase (e.g., 4–6) if soma is thicker
soma_mask = (dist >= soma_min_radius_vox)
soma_mask &= neuron_mask
soma_mask = binary_opening(soma_mask, ball(1))
soma_mask = binary_closing(soma_mask, ball(2))

# (v) Make axis exclusive of soma (final split)
axis_mask = neuron_mask & ~soma_mask

# (vi) Optional: skeleton of neurites
axis_skel = skeletonize_3d(axis_mask)

# (vii) SOMA SHAPE CLASSIFICATION (2D eccentricity on the slice with max soma area)
cell_shape = "none"
soma_cc_lab = label(soma_mask)
if soma_cc_lab.max() > 0:
    props3d = regionprops(soma_cc_lab)
    props3d.sort(key=lambda p: p.area, reverse=True)
    soma_labval = props3d[0].label

    # pick z slice with maximum soma area
    z_areas = [np.sum(soma_cc_lab[z] == soma_labval) for z in range(soma_cc_lab.shape[0])]
    z_best = int(np.argmax(z_areas))

    soma_2d = (soma_cc_lab[z_best] == soma_labval).astype(np.uint8)
    rp2 = regionprops(soma_2d)
    if rp2:
        ecc = rp2[0].eccentricity  # 0≈circle, →1 elongated
        cell_shape = "body" if ecc < 1 else "axon"#0.6,< circular,> elliptical
    else:
        cell_shape = "unknown"

print(f"Soma shape classified as: {cell_shape}")

# --------------------------
# 7) Map lysosomes to neuron/soma/axis and save summaries
# --------------------------
in_neuron = []
in_body = []
in_axis = []
loc_label = []

if len(blobs) > 0:
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not in_bounds(zz, yy, xx, neuron_mask.shape):
            in_neuron.append(False); in_body.append(False); in_axis.append(False)
            loc_label.append("outside")
            continue
        n = bool(neuron_mask[zz, yy, xx])
        s = bool(body_mask[zz, yy, xx])
        a = bool(axis_mask[zz, yy, xx])
        in_neuron.append(n)
        in_body.append(s)
        in_axis.append(a)
        loc_label.append("soma" if s else ("axis" if a else ("neuron_other" if n else "outside")))

if len(df) > 0:
    df["in_neuron_ch2"] = in_neuron
    df["in_body_ch2"]   = in_body
    df["in_axis_ch2"]   = in_axis
    df["location_ch2"]  = loc_label
    df["cell_shape"]    = cell_shape

    # overall counts
    counts = pd.Series({
        "total_lysosomes": len(df),
        "in_neuron": int(np.sum(df["in_neuron_ch2"])),
        "in_body":   int(np.sum(df["in_body_ch2"])),
        "in_axis":   int(np.sum(df["in_axis_ch2"])),
    })
    counts_df = counts.to_frame(name="value")
    counts_df.to_csv("lysosome_location_summary.csv")

    # per-class counts
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_by_location.csv", index=False)

    # detailed table
    df.to_csv("lysosomes_with_neuron_location.csv", index=False)

print("Saved: lysosome_location_summary.csv, lysosome_counts_by_location.csv, lysosomes_with_neuron_location.csv")

# --------------------------
# 8) Visualization: soma & axis overlays + colored lysosomes
# --------------------------
# Soma (red)
soma_layer = viewer.add_labels(soma_mask.astype(np.uint8), name='Body (Ch2)', opacity=0.35)
try:
    soma_layer.color = {1: (1.0, 0.0, 0.0, 1.0)}  # red
except Exception:
    pass
soma_layer.blending = 'translucent_no_depth'

# Axis/neurites (green)
axis_layer = viewer.add_labels(axis_mask.astype(np.uint8), name='Axis (Ch2)', opacity=0.35)
try:
    axis_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
axis_layer.blending = 'translucent_no_depth'

# Optional: skeleton overlay (thin gray/white)
viewer.add_image(
    axis_skel.astype(np.uint8),
    name='Axis skeleton',
    blending='additive',
    contrast_limits=(0, 1),
    colormap='gray',
    opacity=0.8
)

# Lysosomes colored by location (orange=body, cyan=axis, white=outside, gray=neuron_other)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist())
    colors = np.zeros((len(loc), 4), dtype=float)
    colors[loc == "body"]         = [1.0, 0.5, 0.0, 1.0]  # orange
    colors[loc == "axis"]         = [0.0, 1.0, 1.0, 1.0]  # cyan
    colors[loc == "outside"]      = [1.0, 1.0, 1.0, 1.0]  # white
    colors[loc == "neuron_other"] = [0.6, 0.6, 0.6, 1.0]  # gray

    pts = viewer.add_points(
        blobs[:, :3],
        size=np.clip(blobs[:, 3] * 2, 2, None),
        face_color=colors,
        name='Lysosomes (orange=body, cyan=axis)'
    )
    # Add text labels robustly (older/newer napari)
    try:
        pts.properties = {
            'lys_id': df['id'].to_numpy(),
            'where':  df['location_ch2'].to_numpy()
        }
        pts.text = {'text': '{where}', 'size': 8, 'color': 'white', 'anchor': 'upper left'}
        # Optional edges (set after creation for older napari)
        pts.edge_color = 'black'
        pts.edge_width = 0.3
    except Exception:
        pass

# Optional: 3D view
# viewer.dims.ndisplay = 3

# --------------------------
# 9) (Optional) Quick videos
# --------------------------
# Fused overlay (Ch2 base + soma/axis + lysosome circles)
img_norm_2 = to_uint8_stack(image_2)
frames_fused = []
for z in range(img_norm_2.shape[0]):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)

    # Soma in red, Axis in green (simple color overlays)
    soma = (body_mask[z].astype(np.uint8) * 255)
    axis = (axis_mask[z].astype(np.uint8) * 255)
    overlay = base.copy()
    # add red (R channel) for soma
    overlay[..., 2] = np.maximum(overlay[..., 2], body)
    # add green (G channel) for axis
    overlay[..., 1] = np.maximum(overlay[..., 1], axis)
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    # Draw lysosomes by location
    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            # Determine location at this exact voxel
            if in_bounds(z, y, x, body_mask.shape) and body_mask[z, y, x]:
                color = (0, 128, 255)  # BGR orange-ish
            elif in_bounds(z, y, x, axis_mask.shape) and axis_mask[z, y, x]:
                color = (255, 255, 0)  # BGR cyan-ish
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_neuron_soma_axis.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_neuron_soma_axis.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_neuron_soma_axis.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_neuron_soma_axis.gif")

# --------------------------
# 10) RUN VIEWER
# --------------------------
# Put soma shape in the window title (nice reminder)
try:
    viewer.title = "Neuron segmentation — body and axis"
except Exception:
    pass

napari.run()

In [ ]:
###############version 2 but without functios

In [ ]:
# %%
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, meijering, threshold_local
import napari
import pandas as pd
import imageio
import cv2

from skimage.exposure import rescale_intensity
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, skeletonize_3d
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image  = img_ch1       # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (soma + neurites)

# --------------------------
# 2) SMOOTHING (CH1)
# --------------------------
image_smooth = gaussian(image, sigma=1)

# --------------------------
# 3) BLOB DETECTION (CH1)
# --------------------------
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)
if len(blobs) > 0:
    # convert sigma->radius for LoG blobs (3D)
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)
print(f"Detected {len(blobs)} lysosomes.")

# --------------------------
# 4) REGIONAL QUANTIFICATION (CH1) on a 3D GRID (kept; optional)
# --------------------------
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.digitize(blobs[:, 0], z_bins) - 1
    y_idx = np.digitize(blobs[:, 1], y_bins) - 1
    x_idx = np.digitize(blobs[:, 2], x_bins) - 1
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})

if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("radius_voxels", lambda r: np.sum(4/3 * np.pi * r**3))
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 5) NAPARI VIEWER (we’ll overlay Ch1+Ch2 later)
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# --------------------------
# 6) NEURON SEGMENTATION (neuron = soma + axis)
# --------------------------
# Normalize and denoise Ch2 (inline rescale + gaussian)
ch2 = rescale_intensity(image_2.astype(np.float32), in_range='image', out_range=(0,1))
ch2 = gaussian(ch2, sigma=0.6, preserve_range=True)

# (i) Ridge response for thin processes (neurites/axis)
ridge = meijering(ch2, sigmas=np.linspace(0.6, 2.0, 8), black_ridges=False)

# (ii) Base NEURON mask (soma + processes) via local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    # Tune block_size (odd: 51–81) and offset (-0.4..-0.8)*std for your data
    t = threshold_local(R, block_size=61, offset=-0.5*np.std(R))
    neuron_mask[z] = R > t

# cleanup neuron mask
neuron_mask = remove_small_objects(neuron_mask, min_size=8000, connectivity=5)
neuron_mask = binary_closing(neuron_mask, ball(2))

# (iii) AXIS mask proposal from ridge + local threshold
axis_mask_ridge = np.zeros_like(ridge, dtype=bool)
for z in range(ridge.shape[0]):
    Rt = threshold_local(ridge[z], block_size=31, offset=-0.5*np.std(ridge[z]))
    axis_mask_ridge[z] = ridge[z] > Rt

axis_mask_ridge = remove_small_objects(axis_mask_ridge, min_size=200, connectivity=3)
axis_mask_ridge = binary_opening(axis_mask_ridge, ball(1))
axis_mask_ridge = binary_closing(axis_mask_ridge, ball(1))

# Restrict axis to inside neuron
axis_mask = axis_mask_ridge & neuron_mask

# (iv) SOMA via thickness (distance transform): voxels with radius >= threshold
dist = edt(neuron_mask)
soma_min_radius_vox = 3  # <-- key knob; increase (e.g., 4–6) if soma is thicker
soma_mask = (dist >= soma_min_radius_vox)
soma_mask &= neuron_mask
soma_mask = binary_opening(soma_mask, ball(1))
soma_mask = binary_closing(soma_mask, ball(2))

# (v) Make axis exclusive of soma (final split)
axis_mask = neuron_mask & ~soma_mask

# (vi) Optional: skeleton of neurites
axis_skel = skeletonize_3d(axis_mask)

# (vii) SOMA SHAPE CLASSIFICATION (2D eccentricity on the slice with max soma area)
cell_shape = "none"
soma_cc_lab = label(soma_mask)
if soma_cc_lab.max() > 0:
    props3d = regionprops(soma_cc_lab)
    props3d.sort(key=lambda p: p.area, reverse=True)
    soma_labval = props3d[0].label

    # pick z slice with maximum soma area
    z_areas = [np.sum(soma_cc_lab[z] == soma_labval) for z in range(soma_cc_lab.shape[0])]
    z_best = int(np.argmax(z_areas))

    soma_2d = (soma_cc_lab[z_best] == soma_labval).astype(np.uint8)
    rp2 = regionprops(soma_2d)
    if rp2:
        ecc = rp2[0].eccentricity  # 0≈circle, →1 elongated
        cell_shape = "body" if ecc < 1 else "axon"  # 0.6,< circular,> elliptical
    else:
        cell_shape = "unknown"

print(f"Soma shape classified as: {cell_shape}")

# --------------------------
# 7) Map lysosomes to neuron/soma/axis and save summaries
# --------------------------
in_neuron = []
in_body = []
in_axis = []
loc_label = []

if len(blobs) > 0:
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        # inline in_bounds
        if not ((0 <= zz < neuron_mask.shape[0]) and (0 <= yy < neuron_mask.shape[1]) and (0 <= xx < neuron_mask.shape[2])):
            in_neuron.append(False); in_body.append(False); in_axis.append(False)
            loc_label.append("outside")
            continue
        n = bool(neuron_mask[zz, yy, xx])
        s = bool(soma_mask[zz, yy, xx])   # (fixed from body_mask)
        a = bool(axis_mask[zz, yy, xx])
        in_neuron.append(n)
        in_body.append(s)
        in_axis.append(a)
        loc_label.append("soma" if s else ("axis" if a else ("neuron_other" if n else "outside")))

if len(df) > 0:
    df["in_neuron_ch2"] = in_neuron
    df["in_body_ch2"]   = in_body
    df["in_axis_ch2"]   = in_axis
    df["location_ch2"]  = loc_label
    df["cell_shape"]    = cell_shape

    # overall counts
    counts = pd.Series({
        "total_lysosomes": len(df),
        "in_neuron": int(np.sum(df["in_neuron_ch2"])),
        "in_body":   int(np.sum(df["in_body_ch2"])),
        "in_axis":   int(np.sum(df["in_axis_ch2"])),
    })
    counts_df = counts.to_frame(name="value")
    counts_df.to_csv("lysosome_location_summary.csv")

    # per-class counts
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_by_location.csv", index=False)

    # detailed table
    df.to_csv("lysosomes_with_neuron_location.csv", index=False)

print("Saved: lysosome_location_summary.csv, lysosome_counts_by_location.csv, lysosomes_with_neuron_location.csv")

# --------------------------
# 8) Visualization: soma & axis overlays + colored lysosomes
# --------------------------
# Soma (red)
soma_layer = viewer.add_labels(soma_mask.astype(np.uint8), name='Body (Ch2)', opacity=0.35)
try:
    soma_layer.color = {1: (1.0, 0.0, 0.0, 1.0)}  # red
except Exception:
    pass
soma_layer.blending = 'translucent_no_depth'

# Axis/neurites (green)
axis_layer = viewer.add_labels(axis_mask.astype(np.uint8), name='Axis (Ch2)', opacity=0.35)
try:
    axis_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
axis_layer.blending = 'translucent_no_depth'

# Optional: skeleton overlay (thin gray/white)
viewer.add_image(
    axis_skel.astype(np.uint8),
    name='Axis skeleton',
    blending='additive',
    contrast_limits=(0, 1),
    colormap='gray',
    opacity=0.8
)

# Lysosomes colored by location (orange=body, cyan=axis, white=outside, gray=neuron_other)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist())
    colors = np.zeros((len(loc), 4), dtype=float)
    colors[loc == "soma"]         = [1.0, 0.5, 0.0, 1.0]  # orange
    colors[loc == "axis"]         = [0.0, 1.0, 1.0, 1.0]  # cyan
    colors[loc == "outside"]      = [1.0, 1.0, 1.0, 1.0]  # white
    colors[loc == "neuron_other"] = [0.6, 0.6, 0.6, 1.0]  # gray

    pts = viewer.add_points(
        blobs[:, :3],
        size=np.clip(blobs[:, 3] * 2, 2, None),
        face_color=colors,
        name='Lysosomes (orange=soma, cyan=axis)'
    )
    # Add text labels robustly (older/newer napari)
    try:
        pts.properties = {
            'lys_id': df['id'].to_numpy(),
            'where':  df['location_ch2'].to_numpy()
        }
        pts.text = {'text': '{where}', 'size': 8, 'color': 'white', 'anchor': 'upper left'}
        # Optional edges (set after creation for older napari)
        pts.edge_color = 'black'
        pts.edge_width = 0.3
    except Exception:
        pass

# Optional: 3D view
# viewer.dims.ndisplay = 3

# --------------------------
# 9) (Optional) Quick videos
# --------------------------
# Fused overlay (Ch2 base + soma/axis + lysosome circles) — inline min-max to uint8
img_norm_2 = image_2.astype(np.float32)
vmin, vmax = float(img_norm_2.min()), float(img_norm_2.max())
if vmax > vmin:
    img_norm_2 = (img_norm_2 - vmin) / (vmax - vmin)
else:
    img_norm_2[:] = 0.0
img_norm_2 = (img_norm_2 * 255).astype(np.uint8)

frames_fused = []
for z in range(img_norm_2.shape[0]):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)

    # Soma in red, Axis in green (simple color overlays)
    soma = (soma_mask[z].astype(np.uint8) * 255)  # (fixed from body_mask)
    axis = (axis_mask[z].astype(np.uint8) * 255)
    overlay = base.copy()
    # add red (R channel) for soma
    overlay[..., 2] = np.maximum(overlay[..., 2], soma)  # (fixed from 'body')
    # add green (G channel) for axis
    overlay[..., 1] = np.maximum(overlay[..., 1], axis)
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    # Draw lysosomes by location
    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            # Determine location at this exact voxel
            if (0 <= z < soma_mask.shape[0]) and (0 <= y < soma_mask.shape[1]) and (0 <= x < soma_mask.shape[2]) and soma_mask[z, y, x]:
                color = (0, 128, 255)  # BGR orange-ish
            elif (0 <= z < axis_mask.shape[0]) and (0 <= y < axis_mask.shape[1]) and (0 <= x < axis_mask.shape[2]) and axis_mask[z, y, x]:
                color = (255, 255, 0)  # BGR cyan-ish
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_neuron_soma_axis.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_neuron_soma_axis.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_neuron_soma_axis.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_neuron_soma_axis.gif")

# --------------------------
# 10) RUN VIEWER
# --------------------------
try:
    # viewer.title = f"Neuron segmentation — body and axis: {cell_shape}"
    viewer.title = "Neuron segmentation — body and axis"
except (NameError, AttributeError):
    pass

napari.run()

In [ ]:
###VERSION 3###############

In [ ]:
# %%
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, meijering, threshold_local
import napari
import pandas as pd
import imageio
import cv2

from skimage.exposure import rescale_intensity
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, skeletonize_3d
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image  = img_ch1       # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (soma + neurites)

# --------------------------
# 2) CH1: blobs
# --------------------------
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # sigma->radius (3D)
print(f"Detected {len(blobs)} lysosomes.")

# Per-blob DF & region bins (kept as before)
num_bins = (4, 4, 4)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.digitize(blobs[:, 0], z_bins) - 1
    y_idx = np.digitize(blobs[:, 1], y_bins) - 1
    x_idx = np.digitize(blobs[:, 2], x_bins) - 1
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})

if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("radius_voxels", lambda r: np.sum(4/3 * np.pi * r**3))
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 3) Viewer base
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# --------------------------
# 4) CH2: neuron segmentation (neuron = soma + axis)
# --------------------------
ch2 = rescale_intensity(image_2.astype(np.float32), in_range='image', out_range=(0,1))
ch2 = gaussian(ch2, sigma=0.6, preserve_range=True)

ridge = meijering(ch2, sigmas=np.linspace(0.6, 2.0, 8), black_ridges=False)

neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=61, offset=-0.5*np.std(R))
    neuron_mask[z] = R > t

neuron_mask = remove_small_objects(neuron_mask, min_size=8000, connectivity=5)
neuron_mask = binary_closing(neuron_mask, ball(2))

axis_mask_ridge = np.zeros_like(ridge, dtype=bool)
for z in range(ridge.shape[0]):
    Rt = threshold_local(ridge[z], block_size=31, offset=-0.5*np.std(ridge[z]))
    axis_mask_ridge[z] = ridge[z] > Rt

axis_mask_ridge = remove_small_objects(axis_mask_ridge, min_size=200, connectivity=3)
axis_mask_ridge = binary_opening(axis_mask_ridge, ball(1))
axis_mask_ridge = binary_closing(axis_mask_ridge, ball(1))

axis_mask = axis_mask_ridge & neuron_mask

dist = edt(neuron_mask)
soma_min_radius_vox = 3
soma_mask = (dist >= soma_min_radius_vox)
soma_mask &= neuron_mask
soma_mask = binary_opening(soma_mask, ball(1))
soma_mask = binary_closing(soma_mask, ball(2))

axis_mask = neuron_mask & ~soma_mask
axis_skel = skeletonize_3d(axis_mask)

# Soma shape (2D eccentricity on best slice)
soma_shape = "none"
soma_cc_lab = label(soma_mask)
if soma_cc_lab.max() > 0:
    props3d = regionprops(soma_cc_lab)
    props3d.sort(key=lambda p: p.area, reverse=True)
    soma_labval = props3d[0].label
    z_areas = [np.sum(soma_cc_lab[z] == soma_labval) for z in range(soma_cc_lab.shape[0])]
    z_best = int(np.argmax(z_areas))
    soma_2d = (soma_cc_lab[z_best] == soma_labval).astype(np.uint8)
    rp2 = regionprops(soma_2d)
    if rp2:
        ecc = rp2[0].eccentricity
        soma_shape = "circular" if ecc < 0.6 else "elliptical"
    else:
        soma_shape = "unknown"
print(f"Soma shape classified as: {soma_shape}")

# --------------------------
# 5) Map lysosomes to neuron/soma/axis
# --------------------------
in_neuron = []
in_soma = []
in_axis = []
loc_label = []

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            in_neuron.append(False); in_soma.append(False); in_axis.append(False)
            loc_label.append("outside")
            continue
        n = bool(neuron_mask[zz, yy, xx])
        s = bool(soma_mask[zz, yy, xx])
        a = bool(axis_mask[zz, yy, xx])
        in_neuron.append(n)
        in_soma.append(s)
        in_axis.append(a)
        loc_label.append("soma" if s else ("axis" if a else ("neuron_other" if n else "outside")))

if len(df) > 0:
    df["in_neuron_ch2"] = in_neuron
    df["in_soma_ch2"]   = in_soma
    df["in_axis_ch2"]   = in_axis
    df["location_ch2"]  = loc_label
    df["soma_shape"]    = soma_shape

    # overall counts (computed on original column names)
    counts = pd.Series({
        "total_lysosomes": len(df),
        "in_neuron": int(np.sum(df["in_neuron_ch2"])),
        "in_soma":   int(np.sum(df["in_soma_ch2"])),
        "in_axis":   int(np.sum(df["in_axis_ch2"])),
    })
    counts.to_frame(name="value").to_csv("lysosome_location_summary.csv")

    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_by_location.csv", index=False)

    # ---- EXPORT with your requested column names for THIS dataset ----
    df_export = df.copy()
    df_export = df_export.rename(columns={
        "in_soma_ch2": " CELL AXON",   # as requested (note leading space)
        "in_axis_ch2": "CELL BODY"
    })
    df_export.to_csv("lysosomes_with_neuron_location.csv", index=False)

print("Saved: lysosome_location_summary.csv, lysosome_counts_by_location.csv, lysosomes_with_neuron_location.csv")

# --------------------------
# 6) Visualization
# --------------------------
soma_layer = viewer.add_labels(soma_mask.astype(np.uint8), name='Soma (Ch2)', opacity=0.35)
try:
    soma_layer.color = {1: (1.0, 0.0, 0.0, 1.0)}
except Exception:
    pass
soma_layer.blending = 'translucent_no_depth'

axis_layer = viewer.add_labels(axis_mask.astype(np.uint8), name='Axis (Ch2)', opacity=0.35)
try:
    axis_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}
except Exception:
    pass
axis_layer.blending = 'translucent_no_depth'

viewer.add_image(
    axis_skel.astype(np.uint8),
    name='Axis skeleton',
    blending='additive',
    contrast_limits=(0, 1),
    colormap='gray',
    opacity=0.8
)

if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist())
    colors = np.zeros((len(loc), 4), dtype=float)
    colors[loc == "soma"]         = [1.0, 0.5, 0.0, 1.0]  # orange
    colors[loc == "axis"]         = [0.0, 1.0, 1.0, 1.0]  # cyan
    colors[loc == "outside"]      = [1.0, 1.0, 1.0, 1.0]  # white
    colors[loc == "neuron_other"] = [0.6, 0.6, 0.6, 1.0]  # gray

    pts = viewer.add_points(
        blobs[:, :3],
        size=np.clip(blobs[:, 3] * 2, 2, None),
        face_color=colors,
        name='Lysosomes (orange=soma, cyan=axis)'
    )
    try:
        pts.properties = {
            'lys_id': df['id'].to_numpy(),
            'where':  df['location_ch2'].to_numpy()
        }
        pts.text = {'text': '{where}', 'size': 8, 'color': 'white', 'anchor': 'upper left'}
        pts.edge_color = 'black'
        pts.edge_width = 0.3
    except Exception:
        pass

# --------------------------
# 7) Quick fused video (optional)
# --------------------------
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
img_norm_2 = (vol * 255).astype(np.uint8)

frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    soma = (soma_mask[z].astype(np.uint8) * 255)
    axis = (axis_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 2] = np.maximum(overlay[..., 2], soma)  # red for soma
    overlay[..., 1] = np.maximum(overlay[..., 1], axis)  # green for axis
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < soma_mask.shape[1] and 0 <= x < soma_mask.shape[2] and soma_mask[z, y, x]:
                color = (0, 128, 255)  # BGR orange-ish
            elif 0 <= y < axis_mask.shape[1] and 0 <= x < axis_mask.shape[2] and axis_mask[z, y, x]:
                color = (255, 255, 0)  # BGR cyan-ish
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_neuron_soma_axis.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_neuron_soma_axis.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_neuron_soma_axis.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_neuron_soma_axis.gif")

# --------------------------
# 8) Run viewer
# --------------------------
try:
    viewer.title = f"Neuron segmentation — soma shape: {soma_shape}"
except Exception:
    pass

napari.run()

In [ ]:
######Version 4

In [ ]:
# %%
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, meijering, threshold_local
import napari
import pandas as pd
import imageio
import cv2

from skimage.exposure import rescale_intensity
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, skeletonize_3d
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image  = img_ch1       # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (axon + cell body)

# --------------------------
# 2) CH1: blobs
# --------------------------
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # sigma->radius (3D)
print(f"Detected {len(blobs)} lysosomes.")

# Per-blob DF & region bins (optional export kept)
num_bins = (4, 4, 4)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.digitize(blobs[:, 0], z_bins) - 1
    y_idx = np.digitize(blobs[:, 1], y_bins) - 1
    x_idx = np.digitize(blobs[:, 2], x_bins) - 1
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})

if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("radius_voxels", lambda r: np.sum(4/3 * np.pi * r**3))
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 3) Viewer base
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# --------------------------
# 4) CH2: neuron segmentation (axon + cell body)
# --------------------------
# Normalize & denoise
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=0.6, preserve_range=True)

# Thin-process ridge response (neurites)
ridge = meijering(ch2, sigmas=np.linspace(0.6, 2.0, 8), black_ridges=False)

# Base neuron mask via local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=61, offset=-0.5*np.std(R))
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=8000, connectivity=5)
neuron_mask = binary_closing(neuron_mask, ball(2))

# Cell-body proposal from ridge + local threshold (restricted to neuron)
cell_body_mask_ridge = np.zeros_like(ridge, dtype=bool)
for z in range(ridge.shape[0]):
    Rt = threshold_local(ridge[z], block_size=31, offset=-0.5*np.std(ridge[z]))
    cell_body_mask_ridge[z] = ridge[z] > Rt
cell_body_mask_ridge = remove_small_objects(cell_body_mask_ridge, min_size=200, connectivity=3)
cell_body_mask_ridge = binary_opening(cell_body_mask_ridge, ball(1))
cell_body_mask_ridge = binary_closing(cell_body_mask_ridge, ball(1))
cell_body_mask = cell_body_mask_ridge & neuron_mask

# Thickness-based AXON (formerly soma): voxels with radius >= threshold, inside neuron
dist = edt(neuron_mask)
axon_min_radius_vox = 3
axon_mask = (dist >= axon_min_radius_vox)
axon_mask &= neuron_mask
axon_mask = binary_opening(axon_mask, ball(1))
axon_mask = binary_closing(axon_mask, ball(2))

# Make cell body exclusive of axon
cell_body_mask = neuron_mask & ~axon_mask

# Skeleton of cell body (optional)
cell_body_skel = skeletonize_3d(cell_body_mask)

# Axon shape classification (2D eccentricity on slice with max axon area)
axon_shape = "none"
axon_cc_lab = label(axon_mask)
if axon_cc_lab.max() > 0:
    props3d = regionprops(axon_cc_lab)
    props3d.sort(key=lambda p: p.area, reverse=True)
    axon_labval = props3d[0].label
    z_areas = [np.sum(axon_cc_lab[z] == axon_labval) for z in range(axon_cc_lab.shape[0])]
    z_best = int(np.argmax(z_areas))
    axon_2d = (axon_cc_lab[z_best] == axon_labval).astype(np.uint8)
    rp2 = regionprops(axon_2d)
    if rp2:
        ecc = rp2[0].eccentricity
        axon_shape = "body" if ecc < 1 else "axon"
    else:
        axon_shape = "unknown"
print(f"Axon shape classified as: {axon_shape}")

# --------------------------
# 5) Map lysosomes to neuron / axon / cell body
# --------------------------
in_neuron = []
in_axon_ch2 = []   # (formerly in_soma_ch2)
in_body_ch2 = []   # (formerly in_axis_ch2)
location_ch2 = []

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            in_neuron.append(False); in_axon_ch2.append(False); in_body_ch2.append(False)
            location_ch2.append("outside")
            continue
        n = bool(neuron_mask[zz, yy, xx])
        ax = bool(axon_mask[zz, yy, xx])
        cb = bool(cell_body_mask[zz, yy, xx])
        in_neuron.append(n)
        in_axon_ch2.append(ax)
        in_body_ch2.append(cb)
        location_ch2.append("axon" if ax else ("cell body" if cb else ("neuron_other" if n else "outside")))

if len(df) > 0:
    df["in_neuron_ch2"] = in_neuron
    df["in_axon_ch2"]   = in_axon_ch2
    df["in_body_ch2"]   = in_body_ch2
    df["location_ch2"]  = location_ch2
    df["axon_shape"]    = axon_shape

    # overall counts
    counts = pd.Series({
        "total_lysosomes": len(df),
        "in_neuron": int(np.sum(df["in_neuron_ch2"])),
        "in_axon":   int(np.sum(df["in_axon_ch2"])),
        "in_body":   int(np.sum(df["in_body_ch2"])),
    })
    counts.to_frame(name="value").to_csv("lysosome_location_summary.csv")

    # per-class counts
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_by_location.csv", index=False)

    # Export with display names
    df_export = df.copy()
    df_export = df_export.rename(columns={
        "in_axon_ch2": " CELL AXON",   # keep leading space
        "in_body_ch2": "CELL BODY"
    })
    df_export.to_csv("lysosomes_with_neuron_location.csv", index=False)

print("Saved: lysosome_location_summary.csv, lysosome_counts_by_location.csv, lysosomes_with_neuron_location.csv")

# --------------------------
# 6) Visualization (renamed layers)
# --------------------------
# Axon (red)
axon_layer = viewer.add_labels(axon_mask.astype(np.uint8), name='Axon (Ch2)', opacity=0.35)
try:
    axon_layer.color = {1: (1.0, 0.0, 0.0, 1.0)}  # red
except Exception:
    pass
axon_layer.blending = 'translucent_no_depth'

# Cell body (green)
cellbody_layer = viewer.add_labels(cell_body_mask.astype(np.uint8), name='Cell body (Ch2)', opacity=0.35)
try:
    cellbody_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cellbody_layer.blending = 'translucent_no_depth'

# Optional skeleton of cell body
viewer.add_image(
    cell_body_skel.astype(np.uint8),
    name='Cell body skeleton',
    blending='additive',
    contrast_limits=(0, 1),
    colormap='gray',
    opacity=0.8
)

# Lysosomes colored by location (orange=axon, cyan=cell body, white=outside, gray=neuron_other)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist())
    colors = np.zeros((len(loc), 4), dtype=float)
    colors[loc == "axon"]         = [1.0, 0.5, 0.0, 1.0]  # orange
    colors[loc == "cell body"]    = [0.0, 1.0, 1.0, 1.0]  # cyan
    colors[loc == "outside"]      = [1.0, 1.0, 1.0, 1.0]  # white
    colors[loc == "neuron_other"] = [0.6, 0.6, 0.6, 1.0]  # gray

    pts = viewer.add_points(
        blobs[:, :3],
        size=np.clip(blobs[:, 3] * 2, 2, None),
        face_color=colors,
        name='Lysosomes (orange=axon, cyan=cell body)'
    )
    try:
        pts.properties = {
            'lys_id': df['id'].to_numpy(),
            'where':  df['location_ch2'].to_numpy()
        }
        pts.text = {'text': '{where}', 'size': 8, 'color': 'white', 'anchor': 'upper left'}
        pts.edge_color = 'black'
        pts.edge_width = 0.3
    except Exception:
        pass

# Title
try:
    viewer.title = f"Neuron segmentation — axon shape: {axon_shape}"
except Exception:
    pass

# --------------------------
# 7) Quick fused video (optional)
# --------------------------
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    axon = (axon_mask[z].astype(np.uint8) * 255)
    body = (cell_body_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 2] = np.maximum(overlay[..., 2], axon)  # red for axon
    overlay[..., 1] = np.maximum(overlay[..., 1], body)  # green for cell body
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = (int(round(b[1])), int(round(b[2])))
            r = max(2, int(round(b[3])))
            if 0 <= y < axon_mask.shape[1] and 0 <= x < axon_mask.shape[2] and axon_mask[z, y, x]:
                color = (0, 128, 255)  # orange-ish (axon)
            elif 0 <= y < cell_body_mask.shape[1] and 0 <= x < cell_body_mask.shape[2] and cell_body_mask[z, y, x]:
                color = (255, 255, 0)  # cyan-ish (cell body)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_axon_cellbody.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_axon_cellbody.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_axon_cellbody.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_axon_cellbody.gif")

# --------------------------
# 8) Run viewer
# --------------------------
napari.run()

In [ ]:
#####version 5 without segmentation axon and body################

In [ ]:
# %%
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, meijering, threshold_local
import napari
import pandas as pd
import imageio
import cv2

from skimage.exposure import rescale_intensity
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, skeletonize_3d
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image  = img_ch1       # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (axon + cell body)

# --------------------------
# 2) CH1: blobs
# --------------------------
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,#0.005
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # sigma->radius (3D)
print(f"Detected {len(blobs)} lysosomes.")

# Per-blob DF & region bins (optional export kept)
num_bins = (4, 4, 4)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.digitize(blobs[:, 0], z_bins) - 1
    y_idx = np.digitize(blobs[:, 1], y_bins) - 1
    x_idx = np.digitize(blobs[:, 2], x_bins) - 1
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})

if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("radius_voxels", lambda r: np.sum(4/3 * np.pi * r**3))
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 3) Viewer base
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# --------------------------
# 4) CH2: neuron segmentation (axon = thin, cell body = thick)  ✅ three-way
# --------------------------
# Normalize & denoise
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=0.6, preserve_range=True)

# Thin-process ridge response (neurites)
ridge = meijering(ch2, sigmas=np.linspace(0.6, 2.0, 8), black_ridges=False)

# (1) Base NEURON mask (axon + body) via local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=81, offset=-0.4*np.std(R))#0.5#61
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=5)#8000
neuron_mask = binary_closing(neuron_mask, ball(3))#2

# (2) CELL BODY (thick) via distance transform
dist = edt(neuron_mask)
cell_body_min_radius_vox = 1   # tune (e.g., 4–6) if bodies are thicker#3
cell_body_mask = (dist >= cell_body_min_radius_vox)
cell_body_mask &= neuron_mask
cell_body_mask = binary_opening(cell_body_mask, ball(3))#1
cell_body_mask = binary_closing(cell_body_mask, ball(4))#2

# (3) AXON (thin) via ridge + local threshold, inside neuron, excluding cell body
axon_candidate = np.zeros_like(ridge, dtype=bool)
for z in range(ridge.shape[0]):
    Rt = threshold_local(ridge[z], block_size=81, offset=-0.4*np.std(ridge[z]))#31#0.5
    axon_candidate[z] = ridge[z] > Rt
axon_candidate = remove_small_objects(axon_candidate, min_size=100, connectivity=5)#200,3
axon_candidate = binary_opening(axon_candidate, ball(1))
axon_candidate = binary_closing(axon_candidate, ball(1))

axon_mask = axon_candidate & neuron_mask & (~cell_body_mask)   # exclusive split

# (4) Optional: skeleton for the axon
axon_skel = skeletonize_3d(axon_mask)

# (5) Body shape (circular/elliptical), measured on cell body
axon_shape = "unknown"   # kept name to match downstream column
body_lab = label(cell_body_mask)
if body_lab.max() > 0:
    props3d = regionprops(body_lab)
    props3d.sort(key=lambda p: p.area, reverse=True)
    lab = props3d[0].label
    z_areas = [np.sum(body_lab[z] == lab) for z in range(body_lab.shape[0])]
    z_best = int(np.argmax(z_areas))
    body_2d = (body_lab[z_best] == lab).astype(np.uint8)
    rp2 = regionprops(body_2d)
    #if rp2:
    #    ecc = rp2[0].eccentricity
    #    axon_shape = "circular" if ecc < 0.6 else "elliptical"

# Sanity
print("neuron voxels:", int(neuron_mask.sum()))
print("cell body voxels:", int(cell_body_mask.sum()))
print("axon voxels:", int(axon_mask.sum()))

# --------------------------
# 5) Map lysosomes to neuron / axon / cell body
# --------------------------
in_neuron = []
in_axon_ch2 = []
in_body_ch2 = []
location_ch2 = []

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            in_neuron.append(False); in_axon_ch2.append(False); in_body_ch2.append(False)
            location_ch2.append("outside")
            continue
        n = bool(neuron_mask[zz, yy, xx])
        ax = bool(axon_mask[zz, yy, xx])
        cb = bool(cell_body_mask[zz, yy, xx])
        in_neuron.append(n)
        in_axon_ch2.append(ax)
        in_body_ch2.append(cb)
        location_ch2.append("axon" if ax else ("cell body" if cb else ("neuron_other" if n else "outside")))

if len(df) > 0:
    df["in_neuron_ch2"] = in_neuron
    df["in_axon_ch2"]   = in_axon_ch2
    df["in_body_ch2"]   = in_body_ch2
    df["location_ch2"]  = location_ch2
    #df["axon_shape"]    = axon_shape

    # overall counts
    counts = pd.Series({
        "total_lysosomes": len(df),
        "in_neuron": int(np.sum(df["in_neuron_ch2"])),
        "in_axon":   int(np.sum(df["in_axon_ch2"])),
        "in_body":   int(np.sum(df["in_body_ch2"])),
    })
    counts.to_frame(name="value").to_csv("lysosome_location_summary.csv")

    # per-class counts
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_by_location.csv", index=False)

    # Export with display names
    df_export = df.copy()
    df_export = df_export.rename(columns={
        "in_axon_ch2": " CELL AXON",   # keep leading space
        "in_body_ch2": "CELL BODY"
    })
    df_export.to_csv("lysosomes_with_neuron_location.csv", index=False)

print("Saved: lysosome_location_summary.csv, lysosome_counts_by_location.csv, lysosomes_with_neuron_location.csv")

# --------------------------
# 6) Visualization
# --------------------------
# Axon (red)
axon_layer = viewer.add_labels(axon_mask.astype(np.uint8), name='Axon (Ch2)', opacity=0.35)
try:
    axon_layer.color = {1: (1.0, 0.0, 0.0, 1.0)}  # red
except Exception:
    pass
axon_layer.blending = 'translucent_no_depth'

# Cell body (green)
cellbody_layer = viewer.add_labels(cell_body_mask.astype(np.uint8), name='Cell body (Ch2)', opacity=0.35)
try:
    cellbody_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cellbody_layer.blending = 'translucent_no_depth'

# Axon skeleton (thin gray)
viewer.add_image(
    axon_skel.astype(np.uint8),
    name='Axon skeleton',
    blending='additive',
    contrast_limits=(0, 1),
    colormap='gray',
    opacity=0.8
)

# Lysosomes colored by location (orange=axon, cyan=cell body, white=outside, gray=neuron_other)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist())
    colors = np.zeros((len(loc), 4), dtype=float)
    colors[loc == "axon"]         = [1.0, 0.5, 0.0, 1.0]  # orange
    colors[loc == "cell body"]    = [0.0, 1.0, 1.0, 1.0]  # cyan
    colors[loc == "outside"]      = [1.0, 1.0, 1.0, 1.0]  # white
    colors[loc == "neuron_other"] = [0.6, 0.6, 0.6, 1.0]  # gray

    pts = viewer.add_points(
        blobs[:, :3],
        size=np.clip(blobs[:, 3] * 2, 2, None),
        face_color=colors,
        name='Lysosomes (orange=axon, cyan=cell body)'
    )
    try:
        pts.properties = {
            'lys_id': df['id'].to_numpy(),
            'where':  df['location_ch2'].to_numpy()
        }
        pts.text = {'text': '{where}', 'size': 8, 'color': 'white', 'anchor': 'upper left'}
        pts.edge_color = 'black'
        pts.edge_width = 0.3
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation body/axon"
except Exception:
    pass

# --------------------------
# 7) Quick fused video (optional)
# --------------------------
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    axon = (axon_mask[z].astype(np.uint8) * 255)
    body = (cell_body_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 2] = np.maximum(overlay[..., 2], axon)  # red for axon
    overlay[..., 1] = np.maximum(overlay[..., 1], body)  # green for cell body
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < axon_mask.shape[1] and 0 <= x < axon_mask.shape[2] and axon_mask[z, y, x]:
                color = (0, 128, 255)  # orange-ish (axon)
            elif 0 <= y < cell_body_mask.shape[1] and 0 <= x < cell_body_mask.shape[2] and cell_body_mask[z, y, x]:
                color = (255, 255, 0)  # cyan-ish (cell body)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_axon_cellbody.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_axon_cellbody.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_axon_cellbody.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_axon_cellbody.gif")

# --------------------------
# 8) Run viewer
# --------------------------
napari.run()